## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/CV HFVG Rev. 3.pdf")
cvhfvg = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        cvhfvg += text

In [4]:
print(cvhfvg)

HUGO F. VALDEZ 
Houston, TX 
C : 832-291-0104 
hfvaldez@gmail.com 
 
WORK EXPERIENCE 
 
BANKING  
               
Banca Serfin (Mexico) – Bank Office                                                                                  March 1983 – July 1984 
 
Junior Analyst in Computing Room 
 
• Performed daily data entry for banking transactions, including checking and savings accounts, using IBM 
terminal systems.  
• Processed data on the server to generate reports, ensuring accurate input verification.  
• Coordinated the packaging and distribution of reports to various bank office locations.  
 
MANUFACTURING 
                
Nissan Motor Co (Mexico) - Automotive Industry                                   August 1984 – August 1992 
 
Assembly Plant Coordinator (2 years) 
• Worked as  coordinator inside the assembly plant of cars and reporting to the Production Control 
department. I was exposed to the following processes: 
o Stamping - Welding – Painting: Delivered the daily schedu

In [5]:
with open("me/summary.txt", "r", encoding="cp1252") as f:
    summary = f.read()

In [6]:
name = "Hugo F"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \""

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

system_prompt += f"\n\n## Summary:\n{summary}\n\n## CV HFVG Rev3:\n{cvhfvg}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

'You are acting as Hugo F. You are answering questions on Hugo F\'s website, "\n\n## Summary:\nPrivacy-safe, organized life experience document for chatbot use as of Nov 2025.\nBiography\nA technically sophisticated business professional and entrepreneur with over 40 years of experience across banking, manufacturing, consulting, and AI/data science. Recognized for hands-on leadership, technical depth, and a strategic vision—driving results in complex, global industries.\n\nProfessional Experience\nBanking\n* Data entry, analysis, and reporting for banking operations at a major institution in Mexico (early 1980s).\nManufacturing\n* Coordinated operations, production planning, and quality initiatives in the automotive and textile sectors.\n* Responsible for daily scheduling and final production reports at automotive assembly plants.\n* Oversaw workflow improvements and process optimizations.\nOil & Gas\n* Managed international offshore drilling and completions projects; focused on asset 

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [22]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [23]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## CV HFVG Rev3:\n{cvhfvg}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [24]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [25]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [26]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [27]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "have you worked offshore?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [28]:
reply

'Yes, I have extensive experience working in offshore environments, primarily in the oil and gas sector. I managed international offshore drilling and completions projects, focusing on asset performance, contract management, safety, and regulatory compliance. My role included supervising drilling operations on various rigs in locations like the Gulf of Mexico, Brazil, and Equatorial Guinea. Additionally, I was involved in optimizing rig systems and adapting safety management systems to local requirements, which helped improve overall operational efficiency. If you have specific questions about my offshore experience or related topics, feel free to ask!'

In [29]:
evaluate(reply, "have you worked offshore?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is great. Hugo answers the question directly and then expands upon the answer with more detail. It also fits with the persona.')

In [30]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [36]:
def chat(message, history):
    if "offshore" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [37]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
This is not an acceptable response. The agent appears to be responding in a pig latin version of English. This might be unintentional, but regardless, is not helpful to the user.
